# Problem set 1

Linear and integer programming problems.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.2-Linear-programs" data-toc-modified-id="1.2-Linear-programs-1">1.2 Linear programs</a></span><ul class="toc-item"><li><span><a href="#Exercise-2" data-toc-modified-id="Exercise-2-1.1">Exercise 2</a></span></li><li><span><a href="#Exercise-6" data-toc-modified-id="Exercise-6-1.2">Exercise 6</a></span></li></ul></li><li><span><a href="#1.3-Integer-programming" data-toc-modified-id="1.3-Integer-programming-2">1.3 Integer programming</a></span><ul class="toc-item"><li><span><a href="#Exercise-1" data-toc-modified-id="Exercise-1-2.1">Exercise 1</a></span></li><li><span><a href="#Exercise-6" data-toc-modified-id="Exercise-6-2.2">Exercise 6</a></span></li><li><span><a href="#Exercise-8" data-toc-modified-id="Exercise-8-2.3">Exercise 8</a></span></li></ul></li><li><span><a href="#Reference" data-toc-modified-id="Reference-3">Reference</a></span></li></ul></div>

In [1]:
# We import the libraries
using Cbc, JuMP

## 1.2 Linear programs

### Exercise 2

*MUCOW (Milk Undertakings, C and O, Waterloo) owns a herd of Holstein cows and a herd of Jersey cows.
For each herd, the total number of litres produced each day, and milk-fat content (as a percentage) are as
follows:*

|  | Litres produced | Percent milk-fat |
| :-: | :-: | :-: |
| Holstein | 500 | 3.7
| Jersey | 250 | 4.9

*The fat is split off and blended again to create various products.
For each product, the volume, required milk-fat percentage, and profit are as follows.
In particular, the milk-fat percentage must exactly equal what is specified.*

|  | Skimmed milk | 2% | Whole milk | Table cream | Whipping cream |
| :-: | :-: | :-: | :-: | :-: | :-: |
| Volume (litres) | 2 | 2 | 2 | 0.6 | 0.3 |
| Milk-fat requirement | 0% | 2% | 4% | 15% | 45% |
| Profit ($) | 0.1 | 0.15 | 0.2 | 0.5 | 1.2 |

***(a)*** *Formulate as an LP the problem of deciding how many items of each type to produce, so that the total profit is maximized.
Don’t worry about fractional numbers of items.
Write your formulation in matrix notation.*

Consider $x_1, \cdots, x_5$ the number of skimmed milk, ..., whipping cream, respectively.
We want to maximize the profit:

$$\text{maximize } 0.1 x_1 + 0.15 x_2 + 0.2 x_3 + 0.5 x_4 + 1.2 x_5$$

If we split the milk and fat, we have $30.75 L$ of fat and $719.25 L$ of "pure" milk.
So we write the constraints for fat and milk separately:

$$0 x_1 + 0.04 x_2 + 0.08 x_3 + 0.09 x_4 + 0.135 x_5 \le 30.75$$

$$2 x_1 + 1.96 x_2 + 1.92 x_3 + 0.51 x_4 + 0.165 x_5 \le 719.25$$

That is, if we solve this linear problem, we have how many of each product we can produce with the possible milk and fat, in the correct proportions milk-fat.

Let's solve this:

In [2]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Number of each product
@variable(m, x[1:5] >= 0)

# Maximize the profit
@objective(m, Max, 0.1x[1] + 0.15x[2] + 0.2x[3] + 0.5x[4] + 1.2x[5])

# Constraint of fat
@constraint(m, 0.04x[2] + 0.08x[3] + 0.09x[4] + 0.135x[5] <= 30.75)
# Constraint of milk
@constraint(m, 2x[1] + 1.96x[2] + 1.92x[3] + 0.51x[4] + 0.165x[5] <= 719.25)

optimize!(m)

println("\nProfit: ", objective_value(m))
for i=1:5
    println("x_$i qty: ", value(x[i]))
end


Profit: 307.4166666666666
x_1 qty: 340.8333333333333
x_2 qty: 0.0
x_3 qty: 0.0
x_4 qty: 0.0
x_5 qty: 227.77777777777774


So the solution is to use all fat for whipping cream and use the rest of the milk to produce skimmed milk.

***(b)*** *MUCOW is told of a regulation change: 'skimmed milk' can now contain anything up to 0.1\% milk-fat, but no more.
How does the formulation of the problem change? Note the resulting formulation should also be an LP.*

We could change the above approach to include a variable with the milk-fat percentage for skimmed milk, but the problem would not be linear.
Instead, we create another problem.
We try to maximize the same objective function, but we include variables $M_1, \cdots, M_5, f_1, \cdots, f_5$ for the "pure" milk and fat used by each kind of product.
We create the restrictions of milk and fat:

$$\sum_{i = 1}^5 M_i \le 719.25, \ \ \sum_{i = 1}^5 f_i \le 30.75$$

The milk-fat requirements must be satisfied:

$$0 \le \frac{f_1}{M_1 + f_1} \le 0.001$$

$$\frac{f_2}{M_2 + f_2} = 0.02, \ \ \frac{f_3}{M_3 + f_3} = 0.04, \ \ \frac{f_4}{M_4 + f_4} = 0.15, \ \ \frac{f_5}{M_5 + f_5} = 0.45$$

We also calculate the number of each product:

$$x_1 = \frac{M_1 + f_1}{2}, \ \ x_2 = \frac{M_2 + f_2}{2}, \ \ x_3 = \frac{M_3 + f_3}{2}, \ \ x_4 = \frac{M_4 + f_4}{0.6}, \ \ x_5 = \frac{M_5 + f_5}{0.3}$$

In [3]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Number of each product
@variable(m, x[1:5] >= 0)
# Amount of milk
@variable(m, M[1:5] >= 0)
# Amount of fat
@variable(m, f[1:5] >= 0)

# Maximize the profit
@objective(m, Max, 0.1x[1] + 0.15x[2] + 0.2x[3] + 0.5x[4] + 1.2x[5])

# Constraint of fat
@constraint(m, sum(f[1:5]) <= 30.75)
# Constraint of milk
@constraint(m, sum(M[1:5]) <= 719.25)
# Milk-fat requirements
@constraint(m, f[1] <= 0.001M[1] + 0.001f[1])
@constraint(m, f[2] == 0.02M[2] + 0.02f[2])
@constraint(m, f[3] == 0.04M[3] + 0.04f[3])
@constraint(m, f[4] == 0.15M[4] + 0.15f[4])
@constraint(m, f[5] == 0.45M[5] + 0.45f[5])
# Number of each product
@constraint(m, 2x[1] == M[1] + f[1])
@constraint(m, 2x[2] == M[2] + f[2])
@constraint(m, 2x[3] == M[3] + f[3])
@constraint(m, 0.6x[4] == M[4] + f[4])
@constraint(m, 0.3x[5] == M[5] + f[5])

optimize!(m)

println("\nProfit: ", objective_value(m))
for i=1:5
    println("x_$i qty: ", value(x[i]))
end
println("Milk-fat skimmed milk: ", value(f[1])/(value(f[1]) + value(M[1])))


Profit: 307.41666666666663
x_1 qty: 340.8333333333333
x_2 qty: 0.0
x_3 qty: 0.0
x_4 qty: 0.0
x_5 qty: 227.77777777777777
Milk-fat skimmed milk: 0.0


The solution to the last exercise was to use all fat for whipping cream and use the rest of the milk to produce skimmed milk.
The same here!

With respect to the last exercise, we allowed the sollution to give some fat to skimmed milk.
For each whipping cream, it is necessary $0.135 L$ of fat to gain $\$1.2$.
It is not worth it to use $0.1 L$ of fat to gain $\$0.1$ with skimmed milk.

### Exercise 6

We are given an $m$ by $n$ matrix $A$ and a vector $b$ in $\mathbb{R}^m$, for which the system $Ax = b$ has no solution.
Here is an example:

$$\begin{cases}
    2x_1 - x_2 = -1 \\
    x_1 + x_2 = 1 \\
    x_1 + 3x_2 = 4 \\
    -2x_1 + 4x_2 = 3
\end{cases}$$

We are interested in finding a vector $x \in \mathbb{R}^n$ that "comes close" to solving the system.
Namely, we want to find an $x \in \mathbb{R}^n$ whose *deviation* is mininum, and where the deviation of $x$ is defined to be

$$\sum_{i=1}^n\left|b_i - \sum_{j=1}^na_{ij}x_j\right|$$

(For the example system above, the vector $x = (1, 1)^\intercal$ has deviation $2 + 1 + 0 + 1 = 4$.)

***(a)*** *Show that a solution to the optimization problem*

$$\begin{aligned}
    &\text{minimize} & & \sum_{i=1}^{m} y_i \\
    &\text{subject to} & & \\
    & & & \left|\sum_{j=1}^n a_{ij} x_j - b_i\right| \le y_i, \ \ \ \ i = 1, \cdots, m
\end{aligned}$$

*will give a vector $x$ of minimum deviation.*

Note that 

$$\text{minimize} \ \sum_{i=1}^n\left|b_i - \sum_{j=1}^na_{ij}x_j\right|$$

is the same as

$$\begin{aligned}
    &\text{minimize} & & \sum_{i=1}^{m} y_i \\
    &\text{subject to} & & \\
    & & & \left|\sum_{j=1}^n a_{ij} x_j - b_i\right| = y_i, \ \ \ \ i = 1, \cdots, m
\end{aligned}$$

which has the same optimal solution (when it exists) as

$$\begin{aligned}
    &\text{minimize} & & \sum_{i=1}^{m} y_i \\
    &\text{subject to} & & \\
    & & & \left|\sum_{j=1}^n a_{ij} x_j - b_i\right| \le y_i, \ \ \ \ i = 1, \cdots, m
\end{aligned}$$

***(b)*** *The problem of part **(a)** is not a LP.
(Why?)
Show that it can be formulated as an LP.*

The problem of **(a)** is not LP because of the absolute value.
But it is simple to make it LP:

$$\begin{aligned}
&\text{minimize} & & \sum_{i=1}^{m} y_i \\
&\text{subject to} & & \\
& & & \sum_{j=1}^n a_{ij} x_j - b_i \le y_i, \ \ \ \ i = 1, \cdots, m \\
& & & -y_i \le \sum_{j=1}^n a_{ij} x_j - b_i, \ \ \ \ i = 1, \cdots, m
\end{aligned}$$

***(c)*** *Suppose that we had instead defined the deviation of $x$ as*

$$\max_{1 \le i \le m} \left| b_i - \sum_{j = 1}^n a_{ij} x_j \right|$$

*(According to this definition, in the example above $x = (1, 1)^\intercal$ would have deviation $\max\{2, 1, 0, 1\} = 2$.)
With this new definition, write the problem of finding a vector of minimum deviation as an optimization problem, and show that this problem can also be formulated as an LP.*

The problem can be formulated as

$$\text{minimize} \ \max_{1 \le i \le m} \left| b_i - \sum_{j = 1}^n a_{ij} x_j \right|$$

When can do a trick in order to make it LP:

$$\begin{aligned}
    &\text{minimize} & & y \\
    &\text{subject to} & & \\
    & & & \left| b_i - \sum_{j = 1}^n a_{ij} x_j \right| \le y, \ \ \ \ i = 1, \cdots, m
\end{aligned}$$

## 1.3 Integer programming

### Exercise 1

*You are about to trek across the desert with a vehicle having $3.6$ cubic metres ($3.6 \text{m}^3$) of cargo space for goods.
There are various types of items available for putting in this space, each with a different volume and a different net value for your trip, shown as follows:*

| Item type $i$ | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| Volume $v_i$ ($\text{m}^3$) | 0.55 | 0.6 | 0.7 | 0.75 | 0.85 | 0.9 | 0.95 |
| Net value $n_i$ | 250 | 300 | 500 | 700 | 750 | 900 | 950 |

***(a)*** *You need to decide which items to take, not exceeding the volume constraint.
You can take at most one of any item.
No item can be split into fractions.
The total net value must be maximized.
Formulate this problem as an LP or IP.
(You may use the notation $v_i$ and $n_i$ for volume and net value of item $i$.)*

Let $x_1, \cdots, x_7$ be binnary variables indicating if the item $i$ is taken.
So we want to

$$\begin{aligned}
    &\text{maximize} & & \sum_{i = 1}^7 n_i x_i \\
    &\text{subject to} & & \\
    & & & \sum_{i = 1}^7 v_i x_i \le 3.6
\end{aligned}$$

Let's see the implementation in Julia:

In [4]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

n = [250, 300, 500, 700, 750, 900, 950]
v = [0.55, 0.6, 0.7, 0.75, 0.85, 0.9, 0.95]

# Indicator if item is in the car
@variable(m, x[1:7], Bin)

# Maximize the net value
@objective(m, Max, sum(n.*x))

# Constraint of volume
@constraint(m, sum(v.*x) <= 3.6)

optimize!(m)

println("Net value: ", objective_value(m))
for i=1:7
    println("x_$i qty: ", value(x[i]))
end

Net value: 3300.0
x_1 qty: 0.0
x_2 qty: 0.0
x_3 qty: 0.0
x_4 qty: 1.0
x_5 qty: 1.0
x_6 qty: 1.0
x_7 qty: 1.0


***(b)*** *Your two friends have decided to come as well, each with an identical vehicle.
There are exactly two items of each type.
The question is, can you fit all 14 items in the vehicles without exceeding the volume constraints?
No cutting items into pieces is permitted!
Each item taken must be placed entirely in one of the vehicles.
Formulate an LP or IP that has a feasible solution if and only if the items can be packed as desired.
Describe how to determine from a feasible solution how to pack the items into the vehicles.
Note that net value is ignored for part **(b)**.*

We start with a generic objetive function of $\text{maximize } 0$, because we are only looking for feasible solutions.

Let $x_1, \cdots, x_7 \in \{0, 1, 2\}$ be integer variables indicating how many items of each type are in car $1$.
Similarly, $y_1, \cdots, y_7 \in \{0, 1, 2\}$ for car $2$ and $z_1, \cdots, z_7 \in \{0, 1, 2\}$ for car $3$.
So, for each $i = 1, \cdots, 7$, we have that $x_i + y_i + z_i = 2$, that is, there are exactly two items of each type.

We also add the volume constraints:

$$\sum_{i = 1}^7 v_i x_i \le 3.6, \ \ \ \ \sum_{i = 1}^7 v_i y_i \le 3.6, \ \ \ \ \sum_{i = 1}^7 v_i z_i \le 3.6$$

Let's see if the problem is feasible:

In [5]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Indicator of how many items are in car 1
@variable(m, 0 <= x[1:7] <= 2, Int)
# Indicator of how many items are in car 2
@variable(m, 0 <= y[1:7] <= 2, Int)
# Indicator of how many items are in car 3
@variable(m, 0 <= z[1:7] <= 2, Int)

# Without objective function
# We are looking for feasibility only

# Constraint of volume
@constraint(m, sum(v.*x) <= 3.6)
@constraint(m, sum(v.*y) <= 3.6)
@constraint(m, sum(v.*z) <= 3.6)

# For each item, there must be two of them
for i in 1:7
    @constraint(m, x[i] + y[i] + z[i] == 2)
end

optimize!(m)

for i=1:7
    println("x_$i qty: ", value(x[i]))
    println("y_$i qty: ", value(y[i]))
    println("z_$i qty: ", value(z[i]))
end

x_1 qty: 0.0
y_1 qty: 0.0
z_1 qty: 2.0
x_2 qty: 2.0
y_2 qty: 0.0
z_2 qty: 0.0
x_3 qty: 1.0
y_3 qty: 0.0
z_3 qty: 1.0
x_4 qty: 2.0
y_4 qty: 0.0
z_4 qty: 0.0
x_5 qty: 0.0
y_5 qty: 1.0
z_5 qty: 1.0
x_6 qty: 0.0
y_6 qty: 2.0
z_6 qty: 0.0
x_7 qty: 0.0
y_7 qty: 1.0
z_7 qty: 1.0


The problem seems to be feasible.

### Exercise 6

*Consider an LP with variables $x_1, x_2, x_3, x_4$.
Suppose that the LP includes the constraints $x_1, x_2, x_3, x_4 \ge 0$.*

***(a)*** *Consider the constraint:*

$$x_4 \ge |x_3 − 2x_1|.$$

*Suppose that we want to add to the LP the condition that the constraint is satisfied.
Show how to satisfy this requirement so that the resulting formulation is a LP.*

*Hint: rewrite the constraint as a pair of linear inequalities.*

The pair of inequalities is

$$x_4 \ge x_3 − 2x_1, \ \ \ \ - x_4 \le x_3 − 2x_1$$

***(b)*** *Consider the following inequalities:*

$$6x_1 + 2x_2 + 3x_3 + 3x_4 \ge 3,$$

$$2x_1 + 4x_2 + 2x_3 + 7x_4 \ge 9.$$

*Suppose that we want to add to an IP the condition that at least one of
constraints above is satisfied.
Show how to satisfy this requirement so that the resulting formulation is an IP.*

*Hint: add a binary variable indicating whether the above constraints must be satisfied.
Note that the left-hand side of either constraint is always nonnegative.*

We can add the binnary variables $y_1, y_2 \in \{0, 1\}$ so that

$$6x_1 + 2x_2 + 3x_3 + 3x_4 \ge 3 y_1,$$

$$2x_1 + 4x_2 + 2x_3 + 7x_4 \ge 9 y_2.$$

If $y_i$ is $0$, the constraint is useless (left-hand side nonnegative).
If $y_i$ is $1$, the constraint is added, as required.

We also ask that at least one of the constraints is satisfied:

$$y_1 + y_2 \ge 1.$$

****(c)**** *Suppose that for $i = 1, \cdots, k$ we have a non-negative vector $a^i$ with
four entries and a number $\beta_i$ (both $a^i$ and $\beta_i$ are constants).
Let $r$ be any number between $1$ and $k$.
Consider the following set of inequalities:*

$$a_i^\intercal x \ge \beta_i, \ \ \ \ i = 1, \cdots, k.$$

*We want to add to an IP the condition that at least $r$ of the constraints are satisfied.
Show how to satisfy this requirement so that the resulting formulation is an IP.*

*Hint: add a binary variable for each constraint.*

We can use binnary variables $y_i, \ i = 1, \cdots, k$ to indicate if the constraints are being satisfied.
They indicate are follows:

$$a_i^\intercal x \ge \beta_i y_i, \ \ \ \ i = 1, \cdots, k.$$

So we ask that at least $r$ constraints are activated:

$$\sum_{i = 1}^k y_i \ge r.$$

We know the left-hand side is always nonnegative.
In the case $\beta_i$ is positive, $y_1 = 0$ means the constraint is useless, and $y_1 = 1$ means the constraint is activated.

When $\beta_i$ is non positive, it does not matter the value of $y_i$, because the presented contsraint will always be satified.
So $y_i$ must be forced to be $1$ in these cases: $y_i = 1$, if $\beta_i \le 0$.

***(d)*** *Consider the following set of values:*

$$\mathscr{S} \colon= \{3, 9, 17, 19, 36, 67, 1893\}.$$

*Suppose that we want to add to an IP the condition that the variable $x$ takes only one of the values in $\mathscr{S}$.
Show how to satisfy this requirement so that the resulting formulation is an IP.*

*Hint: add a binary variables for each number in the set $\mathscr{S}$.*

We want $x = 3$ or $x = 9$ or $\cdots$ or $x = 1893$.
We create binnary variables $y_1, \cdots, y_7$ so that

$$\begin{aligned}
    x &= 3 y_1 + \cdots + 1893 y_7, \\
    \sum_{i = 1}^7 y_i &= 1
\end{aligned}$$

### Exercise 8

*A company won a government bid to meet the yearly demands $d_1, d_2, \cdots, d_n$ in the areas $j \in \{1, 2, \cdots, n\}$.
Now the company has to decide where to build its factories and how much of each factory’s output will be shipped to which of these $n$ areas.*

*There are $m$ potential locations for building the factories.
If the company decides to build at location $i \in \{1, 2, \cdots, m\}$, then the fixed cost of building the factory (yearly amortized version) is $f_i$ and the yearly capacity of the factory will be $s_i$.
The cost of transporting one unit of the product from location $i$ to area $j$ is given as $c_{ij}$.*

*Construct an IP whose solution indicates where to build the factories, how many units of product to ship from each factory to each demand area so that the demand is met and the total yearly cost of the company is minimized.*

Let $x_1, \cdots, x_m$ be binnary variables indicating which factories were built.
Also let $y_{ij}$ ($i = 1, \cdots, m$, $j = 1, \cdots, n$) indicates how many products are transported from factory $i$ to area $j$.

We want that the transportation that go out of a factory to be satisfied by its production:

$$\sum_{j = 1}^n y_{ij} \le s_i x_i, \ \ \ \ i = 1, \cdots, m$$

Note that it is also constrained by the fact of the factory is activated or not.

We also want that all areas to have yearly demand met:

$$\sum_{i = 1}^m y_{ij} \ge d_j, \ \ \ \ j = 1, \cdots, n$$

And we want to minimize the cost of building and transportation:

$$\text{Minimize} \ \sum_{i = 1}^m f_i x_i + \sum_{i = 1}^m \sum_{j = 1}^n y_{ij} c_{ij}$$

Putting all togheter:

$$\begin{aligned}
    & \text{Minimize} & & \sum_{i = 1}^m f_i x_i + \sum_{i = 1}^m \sum_{j = 1}^n y_{ij} c_{ij} \\
    & \text{subject to} & & \\
    & & & \sum_{j = 1}^n y_{ij} \le s_i x_i, \ \ \ \ i = 1, \cdots, m \\
    & & & \sum_{i = 1}^m y_{ij} \ge d_j, \ \ \ \ j = 1, \cdots, n \\
    & & & x_1, \cdots, x_m \in \{0, 1\} \\
    & & & y_{ij} \in \{0\} \cup \mathbb{N} \ \ \ \ i = 1, \cdots, m, \ j = 1, \cdots, n \\
\end{aligned}$$

## Reference

```bibtex
@book{guenin2014gentle,
    title={A Gentle Introduction to Optimization},
    author={Guenin, B. and K{\"o}nemann, J. and Tun{\c{c}}el, L.},
    isbn={9781107053441},
    lccn={2014008067},
    url={https://books.google.com.br/books?id=JC4DBAAAQBAJ},
    year={2014},
    publisher={Cambridge University Press}
}
```